In [17]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 1 tensorflow v2.xx에서 v1 사용하기

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow V2 비활성화하고 v1활성화
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


## Tensorflow
- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행 결과를 확인

In [6]:
# 1단계 : tensor(상수 노드 하나) 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# 3단계 : 실제 실행 및 출력
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [8]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2. 세션 생성
sess = tf.Session()
# 3. 세션 실행 & 결과 
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [9]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [11]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [14]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3]) 
sess = tf.Session()
sess.run(w)

array([0.13804592, 2.6595354 , 0.89550036], dtype=float32)

In [15]:
#  변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화
sess.run(w)

array([1.289432], dtype=float32)

# 2 tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [19]:
# tensor 그래프 정의
# 데이터 셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:4.946615219116211, W:[0.5018766], b:[0.0583527]
300번째 cost:0.01246507465839386, W:[1.1293594], b:[0.7059356]
600번째 cost:0.0029412659350782633, W:[1.0628375], b:[0.8571561]
900번째 cost:0.000694012560416013, W:[1.0305235], b:[0.9306128]
1200번째 cost:0.00016376013809349388, W:[1.014827], b:[0.96629465]
1500번째 cost:3.864014433929697e-05, W:[1.0072023], b:[0.98362744]
1800번째 cost:9.118695743381977e-06, W:[1.0034988], b:[0.99204654]
2100번째 cost:2.151853095710976e-06, W:[1.0016997], b:[0.9961363]
2400번째 cost:5.082827669866674e-07, W:[1.0008261], b:[0.99812216]
2700번째 cost:1.203392372417511e-07, W:[1.0004023], b:[0.9990864]
3000번째 cost:2.861401604548064e-08, W:[1.0001963], b:[0.9995545]
3300번째 cost:6.845125799515017e-09, W:[1.0000961], b:[0.9997822]
3600번째 cost:1.6838015426401398e-09, W:[1.0000474], b:[0.99989176]
3900번째 cost:3.949859606056094e-10, W:[1.000023], b:[0.99994767]
4200번째 cost:1.1486515211212023e-10, W:[1.0000126], b:[0.99997187]
4500번째 cost:5.328478283606053e-11, W:[1.00000

In [21]:
# 최종적으로 나온 회귀식 H = W*x + b
W_, b_ = sess.run([W, b])

In [22]:
W_, b_

(array([1.0000087], dtype=float32), array([0.9999811], dtype=float32))

In [23]:
def predict(x):
    return W_[0] * x + b_[0]

In [24]:
predict(5)

6.000024616718292

## 2.2 predict을 위한 placeholder 이용
- placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [30]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x + 1
sess = tf.Session()
sess.run([x, H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [31]:
# tensor 그래프 정의
# 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x, y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,
                                                   y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:12.582934379577637, W:[-0.8103775], b:[1.8251091]
300번째 cost:0.06278061866760254, W:[0.7096891], b:[1.6599452]
600번째 cost:0.014813718385994434, W:[0.8589793], b:[1.3205731]
900번째 cost:0.0034954408183693886, W:[0.9314983], b:[1.1557205]
1200번째 cost:0.0008247688529081643, W:[0.966725], b:[1.0756418]
1500번째 cost:0.00019460899056866765, W:[0.98383653], b:[1.0367433]
1800번째 cost:4.591904871631414e-05, W:[0.9921485], b:[1.0178481]
2100번째 cost:1.0835545253939927e-05, W:[0.99618596], b:[1.0086701]
2400번째 cost:2.557284460635856e-06, W:[0.9981471], b:[1.004212]
2700번째 cost:6.035713226992812e-07, W:[0.99909985], b:[1.0020463]
3000번째 cost:1.4271292059220286e-07, W:[0.9995623], b:[1.0009949]
3300번째 cost:3.386314162412418e-08, W:[0.99978685], b:[1.0004847]
3600번째 cost:8.115894623017539e-09, W:[0.99989545], b:[1.0002373]
3900번째 cost:1.9807468998322975e-09, W:[0.9999487], b:[1.0001175]
4200번째 cost:5.271469372125637e-10, W:[0.99997365], b:[1.0000607]
4500번째 cost:9.331794365019519e-11, W:[0.999

In [34]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([5.9999704], dtype=float32)

## 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조정X)

In [ ]:
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])